# Strategy Optimization Analysis Report (US-019)

This notebook analyzes parameter optimization results from batch optimization runs, providing:
- Baseline vs optimized configuration comparison
- Before/after performance metrics visualization
- Parameter sensitivity analysis
- Top configuration rankings
- Deployment recommendations

## Usage

1. Run an optimization: `python scripts/optimize.py --config ... --run-baseline --export-report`
2. Note the output directory (e.g., `data/optimization/run_20241012_143022`)
3. Update `optimization_run_dir` below
4. Run all cells to generate the analysis

In [ ]:
# Configuration
optimization_run_dir = "../data/optimization/run_20241012_143022"  # Replace with your run directory
output_dir = "../data/reports"  # Output directory for generated plots

import os

os.makedirs(output_dir, exist_ok=True)

print(f"Optimization run: {optimization_run_dir}")
print(f"Output directory: {output_dir}")

## 1. Import Dependencies

In [ ]:
import json
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Dependencies loaded successfully!")

## 2. Load Optimization Artifacts

In [ ]:
opt_dir = Path(optimization_run_dir)

# Load baseline metrics
baseline_path = opt_dir / "baseline_metrics.json"
if baseline_path.exists():
    with open(baseline_path) as f:
        baseline_metrics = json.load(f)
    print(f"✅ Loaded baseline metrics: Sharpe={baseline_metrics.get('sharpe_ratio', 0.0):.2f}")
else:
    baseline_metrics = {}
    print("⚠️  No baseline metrics found")

# Load configurations
configs_path = opt_dir / "configs.json"
if configs_path.exists():
    with open(configs_path) as f:
        configs = json.load(f)
    print(f"✅ Loaded {len(configs)} configurations")
else:
    configs = []
    print("⚠️  No configurations found")

# Load ranked results CSV
ranked_path = opt_dir / "ranked_results.csv"
if ranked_path.exists():
    ranked_df = pd.read_csv(ranked_path)
    print(f"✅ Loaded ranked results: {len(ranked_df)} rows")
else:
    ranked_df = pd.DataFrame()
    print("⚠️  No ranked results found")

# Load optimization summary
summary_path = opt_dir / "optimization_summary.json"
if summary_path.exists():
    with open(summary_path) as f:
        opt_summary = json.load(f)
    print("\n✅ Optimization Summary:")
    print(f"   Strategy: {opt_summary.get('strategy', 'unknown')}")
    print(f"   Symbols: {opt_summary.get('symbols', [])}")
    print(f"   Date range: {opt_summary.get('start_date')} to {opt_summary.get('end_date')}")
    print(f"   Total configs: {opt_summary.get('total_configs', 0)}")
    print(f"   Successful: {opt_summary.get('successful_configs', 0)}")
else:
    opt_summary = {}
    print("⚠️  No optimization summary found")

## 3. Baseline vs Best Configuration Comparison

In [ ]:
if configs:
    best_config = configs[0]

    if best_config.get('metrics'):
        best_metrics = best_config['metrics']
        best_acc = best_config.get('accuracy_metrics', {})

        print("="*70)
        print("BASELINE vs BEST CONFIGURATION")
        print("="*70)
        print(f"\n🏆 Best Config: {best_config['config_id']}")
        print(f"   Rank: #{best_config['rank']}")
        print(f"   Composite Score: {best_config['score']:.3f}")

        print("\n📈 Financial Metrics Comparison:")
        print("-" * 70)

        metrics_comparison = [
            ("Sharpe Ratio",
             baseline_metrics.get('sharpe_ratio', 0.0),
             best_metrics.get('sharpe_ratio', 0.0), ""),
            ("Total Return",
             baseline_metrics.get('total_return', 0.0) * 100,
             best_metrics.get('total_return_pct', 0.0), "%"),
            ("Win Rate",
             baseline_metrics.get('win_rate', 0.0) * 100,
             best_metrics.get('win_rate_pct', 0.0), "%"),
        ]

        for metric_name, baseline_val, best_val, unit in metrics_comparison:
            delta = best_val - baseline_val
            arrow = "↑" if delta > 0 else "↓" if delta < 0 else "→"
            color = "green" if delta > 0 else "red" if delta < 0 else "yellow"
            print(f"{metric_name:20s}: {baseline_val:8.2f}{unit:1s} → {best_val:8.2f}{unit:1s}  ({arrow} {delta:+.2f}{unit:1s})")

        if best_acc and baseline_metrics.get('precision_long') is not None:
            print("\n🎯 Accuracy Metrics Comparison:")
            print("-" * 70)

            acc_comparison = [
                ("Precision (LONG)",
                 baseline_metrics.get('precision_long', 0.0) * 100,
                 best_acc.get('precision_long', 0.0) * 100),
                ("Hit Ratio",
                 baseline_metrics.get('hit_ratio', 0.0) * 100,
                 best_acc.get('hit_ratio', 0.0) * 100),
                ("Recall (LONG)",
                 baseline_metrics.get('recall_long', 0.0) * 100,
                 best_acc.get('recall_long', 0.0) * 100),
            ]

            for metric_name, baseline_val, best_val in acc_comparison:
                delta = best_val - baseline_val
                arrow = "↑" if delta > 0 else "↓" if delta < 0 else "→"
                print(f"{metric_name:20s}: {baseline_val:7.1f}% → {best_val:7.1f}%  ({arrow} {delta:+.1f}pp)")

        print("\n⚙️  Best Configuration Parameters:")
        print("-" * 70)
        for param, value in best_config.get('parameters', {}).items():
            print(f"   {param:30s}: {value}")
        print("="*70)
else:
    print("⚠️  No configurations available")

## 4. Before/After Visualization

In [ ]:
if configs and baseline_metrics and configs[0].get('metrics'):
    best_config = configs[0]

    # Prepare data
    categories = ['Sharpe\nRatio', 'Total Return\n(%)', 'Win Rate\n(%)',
                 'Precision\n(LONG %)', 'Hit Ratio\n(%)']

    baseline_vals = [
        baseline_metrics.get('sharpe_ratio', 0.0),
        baseline_metrics.get('total_return', 0.0) * 100,
        baseline_metrics.get('win_rate', 0.0) * 100,
        baseline_metrics.get('precision_long', 0.0) * 100,
        baseline_metrics.get('hit_ratio', 0.0) * 100,
    ]

    best_vals = [
        best_config['metrics'].get('sharpe_ratio', 0.0),
        best_config['metrics'].get('total_return_pct', 0.0),
        best_config['metrics'].get('win_rate_pct', 0.0),
        best_config.get('accuracy_metrics', {}).get('precision_long', 0.0) * 100,
        best_config.get('accuracy_metrics', {}).get('hit_ratio', 0.0) * 100,
    ]

    # Create comparison chart
    x = np.arange(len(categories))
    width = 0.35

    fig, ax = plt.subplots(figsize=(16, 8))

    bars1 = ax.bar(x - width/2, baseline_vals, width, label='Baseline',
                   color='#E74C3C', alpha=0.8, edgecolor='black', linewidth=1.2)
    bars2 = ax.bar(x + width/2, best_vals, width, label='Best Configuration',
                   color='#27AE60', alpha=0.8, edgecolor='black', linewidth=1.2)

    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                   f'{height:.1f}', ha='center', va='bottom',
                   fontsize=9, fontweight='bold')

    ax.set_xlabel('Metrics', fontsize=13, fontweight='bold', labelpad=10)
    ax.set_ylabel('Value', fontsize=13, fontweight='bold', labelpad=10)
    ax.set_title('Before/After Optimization: Performance Comparison',
                fontsize=16, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=11)
    ax.legend(fontsize=12, loc='upper left', frameon=True, shadow=True)
    ax.grid(True, alpha=0.3, axis='y', linestyle='--')

    plt.tight_layout()

    output_path = Path(output_dir) / "before_after_comparison.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"✅ Saved to: {output_path}")

    plt.show()
else:
    print("⚠️  Insufficient data for visualization")

## 5. Parameter Sensitivity Analysis

In [ ]:
if not ranked_df.empty and 'score' in ranked_df.columns:
    print("="*70)
    print("PARAMETER SENSITIVITY ANALYSIS")
    print("="*70)
    print("\nAnalyzing parameter impact on composite score...\n")

    # Identify parameter columns
    exclude_cols = ['config_id', 'rank', 'score', 'sharpe_ratio', 'total_return',
                   'precision_long', 'recall_long', 'hit_ratio', 'win_rate']
    param_cols = [col for col in ranked_df.columns if col not in exclude_cols]

    if param_cols:
        # Calculate correlations
        correlations = {}
        for param in param_cols:
            if pd.api.types.is_numeric_dtype(ranked_df[param]):
                corr = ranked_df[param].corr(ranked_df['score'])
                if not np.isnan(corr):
                    correlations[param] = corr

        if correlations:
            # Sort by absolute correlation
            sorted_corr = sorted(correlations.items(),
                               key=lambda x: abs(x[1]), reverse=True)

            print("Top Parameters by Impact (Correlation with Score):\n")
            for param, corr in sorted_corr[:8]:
                impact = "Positive (↑ increases score)" if corr > 0 else "Negative (↓ decreases score)"
                strength = "Strong" if abs(corr) > 0.5 else "Moderate" if abs(corr) > 0.3 else "Weak"
                print(f"   {param:30s}: {corr:+.3f}  ({strength}, {impact})")

            # Visualization
            fig, ax = plt.subplots(figsize=(14, max(6, len(sorted_corr) * 0.4)))

            params = [item[0] for item in sorted_corr]
            corr_values = [item[1] for item in sorted_corr]
            colors = ['#27AE60' if c > 0 else '#E74C3C' for c in corr_values]

            bars = ax.barh(params, corr_values, color=colors, alpha=0.7,
                          edgecolor='black', linewidth=1.2)

            ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
            ax.set_xlabel('Correlation with Composite Score',
                         fontsize=12, fontweight='bold')
            ax.set_ylabel('Parameter', fontsize=12, fontweight='bold')
            ax.set_title('Parameter Sensitivity Analysis',
                        fontsize=14, fontweight='bold', pad=15)
            ax.grid(True, alpha=0.3, axis='x', linestyle='--')

            # Value labels
            for bar, value in zip(bars, corr_values, strict=False):
                x_pos = value + (0.02 if value > 0 else -0.02)
                ha = 'left' if value > 0 else 'right'
                ax.text(x_pos, bar.get_y() + bar.get_height()/2,
                       f'{value:.3f}', ha=ha, va='center',
                       fontsize=9, fontweight='bold')

            plt.tight_layout()

            output_path = Path(output_dir) / "parameter_sensitivity.png"
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f"\n✅ Saved to: {output_path}")

            plt.show()
        else:
            print("⚠️  No numeric parameters for correlation")
    else:
        print("⚠️  No parameters found")
else:
    print("⚠️  No ranked data available")

## 6. Top 5 Configurations

In [ ]:
if configs:
    print("="*70)
    print("TOP 5 CONFIGURATIONS")
    print("="*70 + "\n")

    top_5 = configs[:5]

    # Create comparison table
    comparison_data = []
    for cfg in top_5:
        if cfg.get('metrics'):
            row = {
                'Rank': cfg['rank'],
                'Config ID': cfg['config_id'],
                'Score': f"{cfg['score']:.3f}",
                'Sharpe': f"{cfg['metrics'].get('sharpe_ratio', 0.0):.2f}",
                'Return (%)': f"{cfg['metrics'].get('total_return_pct', 0.0):.1f}",
            }

            if cfg.get('accuracy_metrics'):
                row['Precision (%)'] = f"{cfg['accuracy_metrics'].get('precision_long', 0.0) * 100:.1f}"
                row['Hit Ratio (%)'] = f"{cfg['accuracy_metrics'].get('hit_ratio', 0.0) * 100:.1f}"

            comparison_data.append(row)

    if comparison_data:
        top_df = pd.DataFrame(comparison_data)
        print(top_df.to_string(index=False))
        print("\n" + "="*70)

        # Visualization
        scores = [float(cfg['score']) for cfg in top_5 if cfg.get('score')]

        if scores:
            fig, ax = plt.subplots(figsize=(12, 6))

            x = np.arange(len(scores))
            colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(scores)))

            bars = ax.bar(x, scores, color=colors, edgecolor='black',
                         linewidth=1.2, alpha=0.8)

            # Value labels
            for bar, score in zip(bars, scores, strict=False):
                ax.text(bar.get_x() + bar.get_width()/2,
                       bar.get_height() + 0.01,
                       f'{score:.3f}', ha='center', va='bottom',
                       fontsize=10, fontweight='bold')

            ax.set_xlabel('Configuration Rank', fontsize=12, fontweight='bold')
            ax.set_ylabel('Composite Score', fontsize=12, fontweight='bold')
            ax.set_title('Top 5 Configurations by Composite Score',
                        fontsize=14, fontweight='bold', pad=15)
            ax.set_xticks(x)
            ax.set_xticklabels([f"#{i+1}" for i in range(len(scores))],
                              fontsize=11)
            ax.grid(True, alpha=0.3, axis='y', linestyle='--')

            plt.tight_layout()

            output_path = Path(output_dir) / "top_5_configurations.png"
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f"\n✅ Saved to: {output_path}")

            plt.show()
else:
    print("⚠️  No configurations available")

## 7. Deployment Recommendations

In [ ]:
if configs:
    best_config = configs[0]

    print("="*70)
    print("DEPLOYMENT RECOMMENDATIONS")
    print("="*70)
    print("\nBased on optimization results, recommended deployment plan:\n")

    print("📋 Phase 1: Validation (2 weeks)")
    print("   - Deploy best configuration in paper trading mode")
    print("   - Monitor live accuracy metrics via dashboard")
    print("   - Compare vs baseline in real-time")
    print("   - Alert on precision < 65% or Sharpe < 1.3\n")

    print("📋 Phase 2: Gradual Rollout (4 weeks)")
    print("   - Week 1-2: 20% capital allocation")
    print("   - Week 3: 50% capital allocation")
    print("   - Week 4: 100% capital if validation successful\n")

    print("📋 Phase 3: Full Production")
    print("   - Update config.py with validated parameters")
    print("   - Archive baseline configuration")
    print("   - Update dashboard alert thresholds\n")

    print("⚠️  Rollback Plan:")
    print("   If live metrics degrade:")
    print("   1. Revert to baseline config immediately")
    print("   2. Analyze live telemetry for root cause")
    print("   3. Re-run optimization on recent data")
    print("   4. Consider adaptive parameter adjustment\n")

    print("🔧 Recommended Parameters:")
    print(f"   Config ID: {best_config['config_id']}")
    for param, value in best_config.get('parameters', {}).items():
        print(f"   {param:30s}: {value}")

    print("\n" + "="*70)

## 8. Export Analysis Summary

In [ ]:
if configs:
    summary_report = {
        "analysis_timestamp": datetime.now().isoformat(),
        "optimization_run": optimization_run_dir,
        "baseline_metrics": baseline_metrics,
        "best_config": configs[0] if configs else None,
        "total_configs": len(configs),
        "strategy": opt_summary.get('strategy'),
        "symbols": opt_summary.get('symbols'),
        "date_range": {
            "start": opt_summary.get('start_date'),
            "end": opt_summary.get('end_date')
        }
    }

    report_path = Path(output_dir) / "optimization_analysis_summary.json"
    with open(report_path, 'w') as f:
        json.dump(summary_report, f, indent=2, default=str)

    print(f"✅ Exported analysis summary to: {report_path}")
    print("\n📊 Analysis complete! Review generated visualizations and reports.")
else:
    print("⚠️  No data to export")